# Advent of Code 2024

# Puzzle - part 1

**--- Day 3: Mull It Over ---**

"Our computers are having issues, so I have no idea if we have any Chief Historians in stock! You're welcome to check the warehouse, though," says the mildly flustered shopkeeper at the *North Pole Toboggan Rental Shop*. The Historians head out to take a look.

The shopkeeper turns to you. "Any chance you can see why our computers are having issues again?"

The computer appears to be trying to run a program, but its memory (your puzzle input) is **corrupted**. All of the instructions have been jumbled up!

It seems like the goal of the program is just to **multiply some numbers**. It does that with instructions like `mul(X,Y)`, where `X` and `Y` are each 1-3 digit numbers. For instance, `mul(44,46)` multiplies 44 by 46 to get a result of `2024`. Similarly, `mul(123,4)` would multiply `123` by `4`.

However, because the program's memory has been corrupted, there are also many invalid characters that should be **ignored**, even if they look like part of a mul instruction. Sequences like `mul(4*`, `mul(6,9!`, `?(12,34)`, or `mul ( 2 , 4 )` do **nothing**.

For example, consider the following section of corrupted memory:

`x`**`mul(2,4)`**`%&mul[3,7]!@^do_not_`**`mul(5,5)`**`+mul(32,64]then(`**`mul(11,8)mul(8,5)`**`)`

Only the four highlighted sections are real `mul` instructions. Adding up the result of each instruction produces **`161`** `(2*4 + 5*5 + 11*8 + 8*5)`.

Scan the corrupted memory for uncorrupted mul instructions.
**What do you get if you add up all of the results of the multiplications?**

## Input

In [19]:
# Load the input file

with open('input - Day 3.txt', 'r') as file:
    input = file.read()

print(input[:139])

(who()where()''~[how()'&do()why()$;mul(323,598)&/-'}{&-/<do(), '~>[?-mul(933,97)how()?from();}{+mul(864,562):#<*$>mul(63,747)what()mul(514,


## Input Formatting

Right now we just have a text file with values from both lists. What we want is two arrays each containing values from only one list. First let us separate each line so that we can later iterate over each line.

In [20]:
from pprint import pprint

pprint(input[:50])

"(who()where()''~[how()'&do()why()$;mul(323,598)&/-"


## Solution

So this seems rather simple, we just need to construct a proper regex to find all matching `mul(X, Y)` patterns.
My question right now is... can the `mul()` be nested?
For example `mul(X, mul(X, Y))`. If so then this puzzle has just gotten a bit harder.

Anyway, we can use regex to match the `mul()` like so `mul\(\d+,\d+\)` and if we use **groups**, `mul\((\d+),(\d+)\)` then we can directly get the number we need to multiply out.

In [21]:
import re

all_muls = re.findall(r"mul\((\d+),(\d+)\)", input)

pprint(all_muls[:10])

[('323', '598'),
 ('933', '97'),
 ('864', '562'),
 ('63', '747'),
 ('514', '101'),
 ('53', '731'),
 ('899', '858'),
 ('402', '353'),
 ('4', '41'),
 ('505', '942')]


In [22]:
all_products = []

for mul in all_muls:

    product = int(mul[0]) * int(mul[1])
    all_products.append(product)

In [23]:
print(f"The sum of all multiplications is: {sum(all_products)}")

The sum of all multiplications is: 185797128


# Puzzle - Part 2


**--- Part Two ---**

As you scan through the corrupted memory, you notice that some of the conditional statements are also still intact. If you handle some of the uncorrupted conditional statements in the program, you might be able to get an even more accurate result.

There are two new instructions you'll need to handle:

The `do()` instruction **enables** future `mul` instructions.
The `don't()` instruction **disables** future `mul` instructions.
Only the most recent `do()` or `don't()` instruction applies. At the beginning of the program, `mul` instructions are **enabled**.

For example:

`x`**`mul(2,4)`**`&mul[3,7]!^`**`don't()`**`_mul(5,5)+mul(32,64](mul(11,8)un`**`do()`**`?`**`mul(8,5)`**`)`

This corrupted memory is similar to the example from before, but this time the `mul(5,5)` and `mul(11,8)` instructions are **disabled** because there is a `don't()` instruction before them. The other mul instructions function normally, including the one at the end that gets re-**enabled** by a `do()` instruction.

This time, the sum of the results is `48` `(2*4 + 8*5)`.

Handle the new instructions; **what do you get if you add up all of the results of just the enabled multiplications?**

______________________________

I think this is rather simple, we just need to modify our regex with an **or** that will allow it to also match the `do()` and `don't()`.
Then as we iterate through our matches we add a flag that keeps track if `mul()` should be active or not.

This looks hard but is actually very easy.
The full regex looks like this `mul\((\d+),(\d+)\)|(do\(\))|(don't\(\))` but we just added two **ors** `| ( do\(\) ) | ( don't\(\) )` and wrapped them in groups each.

In [24]:
all_matches = re.findall(r"mul\((\d+),(\d+)\)|(do\(\))|(don't\(\))", input)

pprint(all_matches[:10])

[('', '', 'do()', ''),
 ('323', '598', '', ''),
 ('', '', 'do()', ''),
 ('933', '97', '', ''),
 ('864', '562', '', ''),
 ('63', '747', '', ''),
 ('514', '101', '', ''),
 ('', '', 'do()', ''),
 ('53', '731', '', ''),
 ('899', '858', '', '')]


Do notice that the `do()` always be at index `2` while `don't` will always be at index `3`.
And since everything is in order we can just iterate over this.

In [25]:
multiplication_flag = True # We start with multiplication enabled
all_products = []

for match in all_matches:

    # Let's make it a bit easier
    num1 = match[0]
    num2 = match[1]
    do = match[2]
    dont = match[3]

    # Remember that empty string evaluate to False, we can just do ifs on them

    # If we matched a do() or don't()
    if do or dont:
        if do:
            multiplication_flag = True
        else:
            multiplication_flag = False
        
    # If we matched one of the mul()
    else:
        if multiplication_flag:
            product = int(num1) * int(num2)
            all_products.append(product)

In [26]:
print(f"The sum of all multiplications is: {sum(all_products)}")

The sum of all multiplications is: 89798695
